In [53]:
import torch
import torch.nn as nn
import torchvision
import numpy
import matplotlib.pyplot as plt
import json
import sys
import os
from PIL import Image

In [81]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5,0.5,0.5],[0.5,0.5,0.5])
])

In [ ]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model = torchvision.models.vgg16(pretrained=True)

In [47]:
for params in model.parameters():
  params.requires_grad = False

In [ ]:
print(model)

In [49]:
model.classifier.add_module('7', nn.Softmax(dim=1))

In [82]:
with open("/content/sample_data/imagenet_class_index.txt","r") as f:
  lines = f.readlines()

class_idx_dict = {}
for line in lines:
  line = line.strip()
  key , value = line.split(':')
  key = key.strip()
  value = value.strip()
  class_idx_dict[key] = value


class_idx = [class_idx_dict[f"{i}"] for i in class_idx_dict.keys()]

In [ ]:
model.to(device)

In [ ]:
folder_path = '/content/Images'
torch.manual_seed(42)
plt.figure(figsize=(20,5))
for file in os.listdir(folder_path):
  if file.endswith('.jpeg') or file.endswith('.png'):
    image_path = os.path.join(folder_path,file)
    image_orig  = Image.open(image_path)
    image = transform(image_orig)
    image = image.unsqueeze(0)
    image = image.to(device)
    predicted = model(image)
    _, output = torch.max(predicted,dim=1)
    image = image.cpu().numpy().squeeze(0).transpose(1,2,0)
    plt.imshow(image_orig)
    plt.title(f"Predicted : {class_idx[output]}")
    plt.show()